# Project setup

In [1]:
import napari
import zarr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import get_single_volume
from DLC_for_WBFM.utils.projects.utils_project import load_config
from pathlib import Path
from DLC_for_WBFM.utils.projects.utils_project import safe_cd
from DLC_for_WBFM.utils.visualization.visualization_tracks import visualize_tracks
from DLC_for_WBFM.utils.preprocessing.utils_tif import PreprocessingSettings
from DLC_for_WBFM.utils.preprocessing.utils_tif import perform_preprocessing
from DLC_for_WBFM.utils.preprocessing.utils_tif import preprocess_all_frames_using_config
import os
from magicgui import magicgui

%load_ext autoreload
%autoreload 2

In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\charles.fieseler\Anaconda3\envs\DLC-for-WBFM\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [2]:
# Set up project

project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\project_config.yaml"
cfg = load_config(project_path)

num_z = cfg['dataset_params']['num_slices']

project_dir = Path(project_path).parent

with safe_cd(project_dir):
    seg_fname = Path(cfg['subfolder_configs']['segmentation'])
    seg_cfg = dict(load_config(seg_fname))
    
    train_fname = Path(cfg['subfolder_configs']['training_data'])
    train_cfg = dict(load_config(train_fname))
    
    track_fname = Path(cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))
    
    p_fname = train_cfg['preprocessing_config']
    p = PreprocessingSettings.load_from_yaml(p_fname)
    
    cfg['preprocessing_config'] = p_fname

In [3]:
# Import segmentation
fname = seg_cfg['output']['masks']
with safe_cd(project_dir):
    raw_segmentation = zarr.open(fname)

fname = os.path.join('4-traces', 'reindexed_masks.zarr')
with safe_cd(project_dir):
    reindexed_segmentation = zarr.open(fname)

In [21]:
# Import DLC tracks
fname = track_cfg['final_3d_tracks']['df_fname']
with safe_cd(project_dir):
    df = pd.read_hdf(fname)

In [5]:
# Import raw data
fname = cfg['preprocessed_red']
with safe_cd(project_dir):
    red_data = zarr.open(fname)

# Define the GUI and layers

In [6]:
def build_df_of_current_points(viewer: napari.Viewer, neuron_names: list) -> pd.DataFrame:
    new_points = viewer.layers['pts_with_past'].data

    col = pd.MultiIndex.from_product([neuron_names, ['t', 'z', 'x', 'y']])
    df_new = pd.DataFrame(columns=col)

    df_new[(name, 't')] = new_points[:,0]
    df_new[(name, 'z')] = new_points[:,1]
    df_new[(name, 'y')] = new_points[:,2]
    df_new[(name, 'x')] = new_points[:,3]

    df_new.sort_values((name,'t'), inplace=True, ignore_index=True)

    return df_new


def save_df(df, name: str) -> None:
    # Save
    out_fname = os.path.join(project_dir, '3-tracking', 'manual_tracking', f'{name}.h5')
    # df_new.to_hdf(out_fname)

    out_fname = str(Path(out_fname).with_suffix('.csv'))
#     df_old = pd.read_csv(out_fname)
#     df_old[name] = df_new[name]
#     df_old.to_csv(out_fname, mode='a')
    df_new.to_csv(out_fname)  # Just overwrite
    
    print(f"Saved manual annotations for neuron {name} at {out_fname}")

def build_tracks_from_name(name='neuron0'):
    # Just visualize one neuron for now
    # 5 columns: 
    # track_id, t, z, y, x
    coords = ['z', 'y', 'x']
    all_tracks_list = []
    likelihood_thresh = 0.4
    zxy_array = np.array(df[name][coords])
    t_array = np.expand_dims(np.arange(zxy_array.shape[0]), axis=1)
    # Remove low likelihood
    to_keep = df[name]['likelihood'] > likelihood_thresh
    zxy_array = zxy_array[to_keep,:]
    t_array = t_array[to_keep,:]

    all_tracks_list.append(np.hstack([t_array, zxy_array]))

    return np.vstack(all_tracks_list)

In [11]:
# neuron_names = list(df.columns.levels[0])

# @magicgui(call_button='Save Annotations',
#          name={"choices": neuron_names})
# def save_widget(viewer: napari.Viewer, name='neuron0') -> napari.types.LayerDataTuple:
#     df = build_df_of_current_points(viewer, neuron_names)
#     save_df(df, name)
    
#     track_array = build_tracks_from_name(name)
    
#     return (track_array, {'name': 'pts_with_future_and_past'}, 'points')

In [12]:
# w1 = widgets.PushButton(value=True, text='PushButton.text')
# w2 = widgets.CheckBox(value=False, text='CheckBox.text')
# container = widgets.Container(widgets=[w1, w2])
# container.show()

In [13]:
def zoom_using_viewer(viewer: napari.Viewer) -> None:
    # Get current point
    t = viewer.dims.current_step[0]
    tzxy = viewer.layers['pts_with_past'].data[t]
    
    # Zoom to it in XY
    viewer.camera.zoom = 10
    viewer.camera.center = tzxy[1:]
    
    # Zoom in Z
    viewer.dims.current_step = (t, tzxy[1], 0, 0)

    
def change_viewer_time_point(viewer: napari.Viewer, dt: int) -> None:
    # Increment time
    t = viewer.dims.current_step[0] + dt
    tzxy = (t, ) + viewer.dims.current_step[1:]
    viewer.dims.current_step = tzxy
    
    
@magicgui(call_button='Zoom to current point')
def zoom_widget(viewer: napari.Viewer) -> None:
    zoom_using_viewer(viewer)

In [14]:
@magicgui(call_button='Zoom to next point')
def zoom_next_widget(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, 1)
    zoom_using_viewer(viewer)

@magicgui(call_button='Zoom to previous point')
def zoom_previous_widget(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, -1)
    zoom_using_viewer(viewer)

In [15]:
type(zoom_next_widget)

magicgui.widgets._function_gui.FunctionGui

In [16]:
all_tracks_array = build_tracks_from_name('neuron0')
pts_with_future_and_past = all_tracks_array
# pts_only_one_frame = pts_with_past # Not made to edit; only 2d version
points_opt = dict(face_color='blue', size=4)

In [17]:
track_of_point = np.hstack([np.ones((all_tracks_array.shape[0], 1)), all_tracks_array])

# Try 2 for GUI: just make it in QT5

In [6]:
from PyQt5 import QtCore, QtWidgets

In [7]:
 viewer = napari.view_image(red_data, ndisplay=2, opacity=0.5)
# viewer = napari.view_image(reindexed_segmentation, ndisplay=2, opacity=0.5)
# viewer.add_labels(reindexed_segmentation, visible=False)
# viewer.add_labels(raw_segmentation, visible=False)
# viewer.add_points(pts_with_future_and_past, n_dimensional=True, symbol='square', **points_opt)
# viewer.add_points(pts_only_one_frame, edge_color='red', symbol='star', visible=False, **points_opt)
# viewer.add_tracks(track_of_point)

In [8]:
# viewer.add_points(ndim=3, name = "pts_with_future_and_past", n_dimensional=True, symbol='square', **points_opt)
# viewer.add_tracks(np.zeros((1,5)), name = "track_of_point")

In [9]:
# def zoom_using_viewer(viewer: napari.Viewer) -> None:
#     # Get current point
#     t = viewer.dims.current_step[0]
#     tzxy = viewer.layers['pts_with_future_and_past'].data[t]
    
#     # Zoom to it in XY
#     viewer.camera.zoom = 10
#     viewer.camera.center = tzxy[1:]
    
#     # Zoom in Z
#     viewer.dims.current_step = (t, tzxy[1], 0, 0)

In [10]:
from DLC_for_WBFM.gui.utils.manual_annotation import manual_annotation_widget

In [22]:
# Make the widget with my new class and add
main_window = viewer.window._qt_window
output_dir = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-immobilized-long\3-tracking\manual_tracking"

ui = manual_annotation_widget()
# ui.setupUi(main_window, df, output_dir, viewer)
ui.setupUi(df, output_dir, viewer)

# Actually dock
viewer.window.add_dock_widget(ui)
ui.show()

# @viewer.bind_key('.', overwrite=True)
# def zoom_next(viewer: napari.Viewer) -> None:
#     change_viewer_time_point(viewer, 1)
#     zoom_using_viewer(viewer)

# @viewer.bind_key(',', overwrite=True)
# def zoom_previous(viewer: napari.Viewer) -> None:
#     change_viewer_time_point(viewer, -1)
#     zoom_using_viewer(viewer)

     neuron0                                       
           t     z          x          y likelihood
0        0.0  17.0  45.357874  71.882004   0.997100
1        1.0  17.0  45.356854  71.880191   0.996462
2        2.0  17.0  45.393682  71.893656   0.996453
3        3.0  17.0  45.371265  71.897385   0.996421
4        4.0  17.0  45.515534  71.853137   0.995701
...      ...   ...        ...        ...        ...
4495  4495.0  17.0  62.498827  71.339777   0.884234
4496  4496.0   0.0   0.000000   0.000000   0.000000
4497  4497.0   0.0   0.000000   0.000000   0.000000
4498  4498.0  17.0  62.580890  71.383208   0.886086
4499  4499.0  17.0  62.746904  71.336567   0.910911

[4500 rows x 5 columns]
     neuron1                                         
           t     z           x           y likelihood
0        0.0  18.0  218.575275  101.605244   0.998570
1        1.0  18.0  218.574101  101.583130   0.998650
2        2.0  18.0  218.621160  101.539171   0.998522
3        3.0  18.0  218.71448

In [18]:
ui.df

bodyparts neuron0                          neuron1                             \
coords          z    x    y likelihood           x   z           y likelihood   
0             0.0  0.0  0.0   0.997100  218.575275  18  101.605244   0.998570   
1             0.0  0.0  0.0   0.996462  218.574101  18  101.583130   0.998650   
2             0.0  0.0  0.0   0.996453  218.621160  18  101.539171   0.998522   
3             0.0  0.0  0.0   0.996421  218.714480  18  101.607014   0.998662   
4             0.0  0.0  0.0   0.995701  218.719253  18  101.589062   0.998735   
...           ...  ...  ...        ...         ...  ..         ...        ...   
4495          0.0  0.0  0.0   0.884234    0.000000  18    0.000000   0.000000   
4496         17.0  0.0  0.0   0.000000    0.000000  18    0.000000   0.000000   
4497         17.0  0.0  0.0   0.000000    0.000000  18    0.000000   0.000000   
4498          0.0  0.0  0.0   0.886086    0.000000  18    0.000000   0.000000   
4499          0.0  0.0  0.0   0.910911    0.000000  18    0.000000   0.000000   

bodyparts   neuron10             ... neuron95                neuron97  \
coords    likelihood          y  ...        z           x           x   
0           0.999125  39.250693  ...       10  184.106174  215.611679   
1           0.999115  39.105873  ...       10  184.023953  215.574067   
2           0.999068  39.133461  ...       10  184.154914  215.609125   
3           0.999120  39.165695  ...       10  184.290531  215.535726   
4           0.999096  39.103953  ...       10  184.390478  215.564108   
...              ...        ...  ...      ...         ...         ...   
4495        0.787826  41.627384  ...       10  204.205950    0.000000   
4496        0.671934  42.380219  ...       10  204.255167    0.000000   
4497        0.896192  41.911964  ...       10  204.302513  226.502940   
4498        0.752517  41.550687  ...       10  204.272375  226.568107   
4499        0.682074  42.312357  ...       10  204.193364    0.000000   

bodyparts                              neuron98                           
coords             y likelihood   z           x          y likelihood  z  
0          38.431032   0.989419  25  171.822285  70.389160   0.999394  8  
1          38.410458   0.987854  25  171.820814  70.383354   0.999528  8  
2          38.540917   0.988467  25  171.924306  70.357290   0.999553  8  
3          38.547232   0.988214  25  171.961043  70.386206   0.999438  8  
4          38.553925   0.988474  25  172.092634  70.385913   0.999531  8  
...              ...        ...  ..         ...        ...        ... ..  
4495        0.000000   0.000000  25    0.000000   0.000000   0.000000  8  
4496        0.000000   0.000000  25  192.780225  68.148792   0.799803  8  
4497       44.580708   0.471194  25  192.757038  68.148577   0.817601  8  
4498       44.589540   0.410404  25  192.775314  68.189203   0.778942  8  
4499        0.000000   0.000000  25  192.774141  68.206548   0.787221  8  

[4500 rows x 412 columns]

In [19]:
df

bodyparts neuron0                          neuron1                             \
coords          z    x    y likelihood           x   z           y likelihood   
0             0.0  0.0  0.0   0.997100  218.575275  18  101.605244   0.998570   
1             0.0  0.0  0.0   0.996462  218.574101  18  101.583130   0.998650   
2             0.0  0.0  0.0   0.996453  218.621160  18  101.539171   0.998522   
3             0.0  0.0  0.0   0.996421  218.714480  18  101.607014   0.998662   
4             0.0  0.0  0.0   0.995701  218.719253  18  101.589062   0.998735   
...           ...  ...  ...        ...         ...  ..         ...        ...   
4495          0.0  0.0  0.0   0.884234    0.000000  18    0.000000   0.000000   
4496         17.0  0.0  0.0   0.000000    0.000000  18    0.000000   0.000000   
4497         17.0  0.0  0.0   0.000000    0.000000  18    0.000000   0.000000   
4498          0.0  0.0  0.0   0.886086    0.000000  18    0.000000   0.000000   
4499          0.0  0.0  0.0   0.910911    0.000000  18    0.000000   0.000000   

bodyparts   neuron10             ... neuron95                neuron97  \
coords    likelihood          y  ...        z           x           x   
0           0.999125  39.250693  ...       10  184.106174  215.611679   
1           0.999115  39.105873  ...       10  184.023953  215.574067   
2           0.999068  39.133461  ...       10  184.154914  215.609125   
3           0.999120  39.165695  ...       10  184.290531  215.535726   
4           0.999096  39.103953  ...       10  184.390478  215.564108   
...              ...        ...  ...      ...         ...         ...   
4495        0.787826  41.627384  ...       10  204.205950    0.000000   
4496        0.671934  42.380219  ...       10  204.255167    0.000000   
4497        0.896192  41.911964  ...       10  204.302513  226.502940   
4498        0.752517  41.550687  ...       10  204.272375  226.568107   
4499        0.682074  42.312357  ...       10  204.193364    0.000000   

bodyparts                              neuron98                           
coords             y likelihood   z           x          y likelihood  z  
0          38.431032   0.989419  25  171.822285  70.389160   0.999394  8  
1          38.410458   0.987854  25  171.820814  70.383354   0.999528  8  
2          38.540917   0.988467  25  171.924306  70.357290   0.999553  8  
3          38.547232   0.988214  25  171.961043  70.386206   0.999438  8  
4          38.553925   0.988474  25  172.092634  70.385913   0.999531  8  
...              ...        ...  ..         ...        ...        ... ..  
4495        0.000000   0.000000  25    0.000000   0.000000   0.000000  8  
4496        0.000000   0.000000  25  192.780225  68.148792   0.799803  8  
4497       44.580708   0.471194  25  192.757038  68.148577   0.817601  8  
4498       44.589540   0.410404  25  192.775314  68.189203   0.778942  8  
4499        0.000000   0.000000  25  192.774141  68.206548   0.787221  8  

[4500 rows x 412 columns]

# Use the GUI to track

In [44]:
viewer = napari.view_image(red_data, ndisplay=2, opacity=0.5)
# viewer = napari.view_image(reindexed_segmentation, ndisplay=2, opacity=0.5)
# viewer.add_labels(reindexed_segmentation, visible=False)
# viewer.add_labels(raw_segmentation, visible=False)
viewer.add_points(pts_with_future_and_past, n_dimensional=True, symbol='square', **points_opt)
# viewer.add_points(pts_only_one_frame, edge_color='red', symbol='star', visible=False, **points_opt)
viewer.add_tracks(track_of_point)

<Tracks layer 'track_of_point' at 0x27fdeaefbc8>

In [45]:
viewer.window.add_dock_widget(save_widget)
# viewer.window.add_dock_widget(zoom_widget)
# viewer.window.add_dock_widget(zoom_next_widget)

In [13]:
@viewer.bind_key('.', overwrite=True)
def zoom_next(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, 1)
    zoom_using_viewer(viewer)

In [14]:
@viewer.bind_key(',', overwrite=True)
def zoom_previous(viewer: napari.Viewer) -> None:
    change_viewer_time_point(viewer, -1)
    zoom_using_viewer(viewer)

In [25]:
viewer.title

'napari'

In [16]:
viewer.layers['pts_with_past'].data[10]

array([10.        , 17.        , 71.86739407, 45.45017191])

In [17]:
viewer.dims.current_step

(0, 0, 0, 0)

In [48]:
main_window = viewerwindow._qt_window

In [54]:
viewer.layers['track_of_point'].data

array([[1.00000000e+00, 0.00000000e+00, 1.70000000e+01, 8.78200378e+00,
        1.23578739e+01],
       [1.00000000e+00, 1.00000000e+00, 1.70000000e+01, 7.18801910e+01,
        4.53568541e+01],
       [1.00000000e+00, 2.00000000e+00, 1.70000000e+01, 7.18936564e+01,
        4.53936824e+01],
       ...,
       [1.00000000e+00, 4.49500000e+03, 1.70000000e+01, 7.13397774e+01,
        6.24988270e+01],
       [1.00000000e+00, 4.49800000e+03, 1.70000000e+01, 7.13832081e+01,
        6.25808895e+01],
       [1.00000000e+00, 4.49900000e+03, 1.70000000e+01, 7.13365667e+01,
        6.27469041e+01]])

# Next: just look at training data

In [23]:
# Set up project

project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"
cfg = load_config(project_path)

num_z = cfg['dataset_params']['num_slices']

project_dir = Path(project_path).parent

with safe_cd(project_dir):
    seg_fname = Path(cfg['subfolder_configs']['segmentation'])
    seg_cfg = dict(load_config(seg_fname))
    
    train_fname = Path(cfg['subfolder_configs']['training_data'])
    train_cfg = dict(load_config(train_fname))
    
    track_fname = Path(cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))

In [25]:
# Import segmentation
fname = seg_cfg['output']['masks']
with safe_cd(project_dir):
    raw_segmentation = zarr.open(fname)

# fname = os.path.join('4-traces', 'reindexed_masks.zarr')
# with safe_cd(project_dir):
#     reindexed_segmentation = zarr.open(fname)

In [21]:
# Import DLC tracks
fname = track_cfg['final_3d_tracks']['df_fname']
with safe_cd(project_dir):
    df = pd.read_hdf(fname)

In [5]:
# Import raw data
fname = cfg['preprocessed_red']
with safe_cd(project_dir):
    red_data = zarr.open(fname)

# Aside: convert my dataframe format to DLC style (training data)

In [ ]:
project_path = r"Y:\shared_projects\wbfm\dlc_stacks\Charlie-worm3-new-seg\project_config.yaml"

project_cfg = load_config(project_path)
project_dir = Path(project_path).parent

with safe_cd(project_dir):
    track_fname = Path(project_cfg['subfolder_configs']['tracking'])
    track_cfg = dict(load_config(track_fname))

    segment_fname = Path(project_cfg['subfolder_configs']['segmentation'])
    segment_cfg = dict(load_config(segment_fname))

this_config = {'track_cfg':track_cfg, 'segment_cfg': segment_cfg, 'project_cfg':project_cfg}
this_config['dataset_params'] = project_cfg['dataset_params'].copy()

with safe_cd(_config['project_dir']):
    reindex_segmentation_only_training_data(this_config, DEBUG=True)